# Erratum

This notebook provides changes in the code after publication. Fortunately (detected) bug(s) in the printed code do(es) not prevent to do the exercise and perform analysis. Each bug has own description.

## 30.01.2019
## Bug no.1: reprojection of multipart polygons is not correct [SOLVED]

### Description:
Reprojection method in the VectorData class works well only for the single part geometries - like 97% of counties in Poland and many other administrative world borders as well as artificial boundaries (vectors) used to mark the area of interest.
But in some cases this method fails. It is true when polygon consists multiple parts which are not connected. The best example are islands, where one vector file is created from many parts - one for each landmass above the sea level. It is true also for the specific poviats in Poland. Good example is the włocławski poviat presented below (By OpenStreetMap contributors):

<img src="https://upload.wikimedia.org/wikipedia/commons/a/ac/Powiat_w%C5%82oc%C5%82awski_location_map.png" title="Powiat Włocławski. Źródło: Wikipedia">

### Correction:

In [2]:
class VectorData:
    """Class reads, stores, shows and reprojects vector shapefiles.
    It is initialized with filename.
    
    Class methods:
    get_vector_data(parameters='all', write_params=True) - method reads vector data and parse its parameters such as geometry,
    properties, crs and schema into dictionaries. Method may store parameters inside the object if write_params is set
    to True. Method returns desired parameters.
    reproject_geometry(destination_epsg, update=False) - method changes projection of each point inside the Polygon into
    projection given as an epsg number. It may update object geometry if the update parameter is set to True. Method returns
    reprojected geometry dictionary.
    show_vector_data(geometry=None) - method shows polygon. If geometry is not given then method uses class geometry."""
    
    def __init__(self, filename):
        self.v_file = filename
        self.geometry = {}
        self.properties = None
        self.vec_crs = None
        self.vec_schema = None
        
    def get_vector_data(self, parameters='all', write_params=True):
        """
        Function reads vector and its parameters and return them and / or store them inside the objects instance.
        input 1: parameters (string)
        parameters: 'all', 'none', 'geometry', 'properties', 'crs', 'schema'
        'all': returns tuple with geometry, properties, crs and schema;
        'none': does not return anything;
        'geometry', 'properties', 'crs' or 'schema': returns chosen parameter.
        input 2: write_params (bool)
        write_params: True, False
        True: store all parameters in the object
        False: do not store anything in the object
        output: chosen parameters as a dict"""
        
        with fio.open(self.v_file, 'r') as masking_region:
            geometry = [feature["geometry"] for feature in masking_region]
            properties = [feature['properties'] for feature in masking_region]
            vec_crs = masking_region.crs
            vec_schema = masking_region.schema
        
        if write_params:
            print('--- Object geometry, properties, crs and schema updated ---')
            self.geometry = geometry
            self.properties = properties
            self.vec_crs = vec_crs
            self.vec_schema = vec_schema
        else:
            print('--- Object parameters not updated ---')
            
        output_dict = {'all': ({'geometry': geometry, 'properties': properties, 'crs': vec_crs, 'scheme': vec_schema}),
                      'none': 0,
                      'geometry': geometry,
                      'properties': properties,
                      'crs': vec_crs,
                      'schema': vec_schema}
        try:
            return output_dict[parameters]
        except KeyError:
            raise KeyError("Parameter not available. Available parameters: 'all', 'none', 'geometry', 'properties', 'crs', 'schema'")
      
    def reproject_geometry(self, destination_epsg, update=False):
        """
        Function reprojects vector geometry and updates it.
        input 1: destination_epsg (string or int)
        input 2: update (bool)
        True: update object's geometry
        False: do not update object's geometry
        output: reprojected geometry as a dict"""

        try:
            destination_crs = from_epsg(destination_epsg)
            proj_crs_in = Proj(self.vec_crs)
            coordinates_list = self.geometry[0]['coordinates']
        except RuntimeError:
            raise ValueError('Given EPSG is wrong or it is not stored in fiona to run from_epsg method')
        except KeyError:
            error_text = 'Please, update your object parameters ' \
                         'by get_vector_data method with write_params set to True.'
            raise ValueError(error_text)
        proj_crs_out = Proj(init=destination_crs['init'])
        
        
        #### CORRECTED PART STARTS FROM HERE ####
        
        projected_geometries = []

        for geometries in coordinates_list:
            projected_g = []

            # Check if it is a multipart geometry
            if len(geometries) == 1:
                for g in geometries[0]:
                    transformed = transform(proj_crs_in, proj_crs_out, g[0], g[1])
                    points = (transformed[0], transformed[1],)
                    projected_g.append(points)
            else:
                for g in geometries:
                    transformed = transform(proj_crs_in, proj_crs_out, g[0], g[1])
                    points = (transformed[0], transformed[1],)
                    projected_g.append(points)
            projected_geometries.append(projected_g)
        geometry_dict = {'coordinates': projected_geometries, 'type': 'Polygon'}
        
        #### CORRECTED PART ENDS HERE ####
        
        if update:
            self.geometry = [geometry_dict]
            self.vec_crs = destination_crs

        return [geometry_dict]
    
    def show_vector_data(self, geometry=None):
        """
        Function shows vector geometry.
        input 1: geometry (dict, geometry retrieved as a parameter from the vector data type).
        If geometry is not given then method takes it from the objects instance."""
        
        if geometry:
            g = geometry
        else:
            g = self.geometry
            
        try:
            coordinates_list = g[0]['coordinates'][0]
        except KeyError:
            raise ValueError(
                'Geometry is not defined. Please, update geometry with get_vector_data method or provide valid geometry object to the method.')
            
        coordinates_array = np.asarray(coordinates_list)
        plt.figure()
        plt.plot(coordinates_array[:, 0], coordinates_array[:, 1])
        plt.show()

#### CORRECT VERSION IMPLEMENTED IN THE SOLUTION.IPYNB FILE